<a href="https://colab.research.google.com/github/Shaheer-Khan/AISem3/blob/master/HW/Home_Credit_default_risk_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install xgboost

In [ ]:
#data.set_index("SK_ID_CURR",
#               drop=True,
#               inplace=True)

#test.set_index("SK_ID_CURR",
#               drop=True,
#               inplace=True)

#full_set = pd.concat([data, test])

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shaheersk","key":"7d251b05c20db9fd4c81f313f4770bcd"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip uninstall -y kaggle
!pip install —upgrade pip
!pip install kaggle==1.5.6
!kaggle -v

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
ERROR: Invalid requirement: '—upgrade'
     |████████████████████████████████| 61kB 2.2MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=22da01d62f836f9d67c688c4361c04f8a68a16c0736ce4fed7cffa96e6b4a7b7
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
Kaggle API 1.5.6


In [ ]:
!kaggle competitions download -c home-credit-default-risk

100% 688M/688M [00:26<00:00, 42.9MB/s]
100% 688M/688M [00:26<00:00, 26.8MB/s]


In [ ]:
from zipfile import ZipFile

file_name = "home-credit-default-risk.zip"

with ZipFile(file_name,"r") as zip:
  zip.extractall()
  print('done')

done


In [1]:
import numpy as np 
import pandas as pd

In [2]:
train = pd.read_csv('application_train.csv')
bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
credit_card_balance = pd.read_csv('credit_card_balance.csv')
installments_payments = pd.read_csv('installments_payments.csv')
previous_application = pd.read_csv('previous_application.csv')
POS_CASH_balance = pd.read_csv('POS_CASH_balance.csv')

test = pd.read_csv('application_test.csv')
sample_sub = pd.read_csv('sample_submission.csv')

In [3]:
print('Train data', train.shape)
print('bureau data', bureau.shape)
print('bureau_balance data', bureau_balance.shape)
print('previous_application data', previous_application.shape)
print('POS_CASH_balance data', POS_CASH_balance.shape)
print('installments_payments data', installments_payments.shape)
print('credit_card_balance data', credit_card_balance.shape)
print('Test data shape: ', test.shape)

Train data (307511, 122)
bureau data (1716428, 17)
bureau_balance data (27299925, 3)
previous_application data (1670214, 37)
POS_CASH_balance data (10001358, 8)
installments_payments data (13605401, 8)
credit_card_balance data (3840312, 23)
Test data shape:  (48744, 121)


In [ ]:
train['TARGET'].value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [ ]:
train.select_dtypes('float64').apply(pd.Series.nunique, axis = 0)

AMT_INCOME_TOTAL               2548
AMT_CREDIT                     5603
AMT_ANNUITY                   13672
AMT_GOODS_PRICE                1002
REGION_POPULATION_RELATIVE       81
                              ...  
AMT_REQ_CREDIT_BUREAU_DAY         9
AMT_REQ_CREDIT_BUREAU_WEEK        9
AMT_REQ_CREDIT_BUREAU_MON        24
AMT_REQ_CREDIT_BUREAU_QRT        11
AMT_REQ_CREDIT_BUREAU_YEAR       25
Length: 65, dtype: int64

In [ ]:
train.select_dtypes('int64').apply(pd.Series.nunique, axis = 0)

SK_ID_CURR                     307511
TARGET                              2
CNT_CHILDREN                       15
DAYS_BIRTH                      17460
DAYS_EMPLOYED                   12574
DAYS_ID_PUBLISH                  6168
FLAG_MOBIL                          2
FLAG_EMP_PHONE                      2
FLAG_WORK_PHONE                     2
FLAG_CONT_MOBILE                    2
FLAG_PHONE                          2
FLAG_EMAIL                          2
REGION_RATING_CLIENT                3
REGION_RATING_CLIENT_W_CITY         3
HOUR_APPR_PROCESS_START            24
REG_REGION_NOT_LIVE_REGION          2
REG_REGION_NOT_WORK_REGION          2
LIVE_REGION_NOT_WORK_REGION         2
REG_CITY_NOT_LIVE_CITY              2
REG_CITY_NOT_WORK_CITY              2
LIVE_CITY_NOT_WORK_CITY             2
FLAG_DOCUMENT_2                     2
FLAG_DOCUMENT_3                     2
FLAG_DOCUMENT_4                     2
FLAG_DOCUMENT_5                     2
FLAG_DOCUMENT_6                     2
FLAG_DOCUMEN

In [ ]:
train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

NAME_CONTRACT_TYPE             2
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
WALLSMATERIAL_MODE             7
EMERGENCYSTATE_MODE            2
dtype: int64

no null values in vraiables with dtypes int64

In [4]:
train.select_dtypes('object').isnull().sum()

NAME_CONTRACT_TYPE                 0
CODE_GENDER                        0
FLAG_OWN_CAR                       0
FLAG_OWN_REALTY                    0
NAME_TYPE_SUITE                 1292
NAME_INCOME_TYPE                   0
NAME_EDUCATION_TYPE                0
NAME_FAMILY_STATUS                 0
NAME_HOUSING_TYPE                  0
OCCUPATION_TYPE                96391
WEEKDAY_APPR_PROCESS_START         0
ORGANIZATION_TYPE                  0
FONDKAPREMONT_MODE            210295
HOUSETYPE_MODE                154297
WALLSMATERIAL_MODE            156341
EMERGENCYSTATE_MODE           145755
dtype: int64

In [ ]:
((train.select_dtypes('object').isnull().sum())/train.select_dtypes('object').shape[0])*100

NAME_CONTRACT_TYPE             0.000000
CODE_GENDER                    0.000000
FLAG_OWN_CAR                   0.000000
FLAG_OWN_REALTY                0.000000
NAME_TYPE_SUITE                0.420148
NAME_INCOME_TYPE               0.000000
NAME_EDUCATION_TYPE            0.000000
NAME_FAMILY_STATUS             0.000000
NAME_HOUSING_TYPE              0.000000
OCCUPATION_TYPE               31.345545
WEEKDAY_APPR_PROCESS_START     0.000000
ORGANIZATION_TYPE              0.000000
FONDKAPREMONT_MODE            68.386172
HOUSETYPE_MODE                50.176091
WALLSMATERIAL_MODE            50.840783
EMERGENCYSTATE_MODE           47.398304
dtype: float64

In [ ]:
train['NAME_CONTRACT_TYPE'].unique()

array(['Cash loans', 'Revolving loans'], dtype=object)

In [ ]:
train['ORGANIZATION_TYPE'].unique()

array(['Business Entity Type 3', 'School', 'Government', 'Religion',
       'Other', 'XNA', 'Electricity', 'Medicine',
       'Business Entity Type 2', 'Self-employed', 'Transport: type 2',
       'Construction', 'Housing', 'Kindergarten', 'Trade: type 7',
       'Industry: type 11', 'Military', 'Services', 'Security Ministries',
       'Transport: type 4', 'Industry: type 1', 'Emergency', 'Security',
       'Trade: type 2', 'University', 'Transport: type 3', 'Police',
       'Business Entity Type 1', 'Postal', 'Industry: type 4',
       'Agriculture', 'Restaurant', 'Culture', 'Hotel',
       'Industry: type 7', 'Trade: type 3', 'Industry: type 3', 'Bank',
       'Industry: type 9', 'Insurance', 'Trade: type 6',
       'Industry: type 2', 'Transport: type 1', 'Industry: type 12',
       'Mobile', 'Trade: type 1', 'Industry: type 5', 'Industry: type 10',
       'Legal Services', 'Advertising', 'Trade: type 5', 'Cleaning',
       'Industry: type 13', 'Trade: type 4', 'Telecom',
       'I

As the secondary tables have many to one relationship with the main table (application train/test) hierarchy is: <br>
1. application train/test | key = SK_ID_CURR to | (bureau and previous_application) <br>
2. bureau | key = SK_ID_BUREAU to | bureau_balance <br>
3. previous_application | key = SK_ID_PREV to | (POS_CASH_balance, installments_payments and credit_card_balance) <br> <br>
Merge the tables

2. bureau_merge

In [ ]:
bureau.dtypes

SK_ID_CURR                  int64
SK_ID_BUREAU                int64
CREDIT_ACTIVE              object
CREDIT_CURRENCY            object
DAYS_CREDIT                 int64
CREDIT_DAY_OVERDUE          int64
DAYS_CREDIT_ENDDATE       float64
DAYS_ENDDATE_FACT         float64
AMT_CREDIT_MAX_OVERDUE    float64
CNT_CREDIT_PROLONG          int64
AMT_CREDIT_SUM            float64
AMT_CREDIT_SUM_DEBT       float64
AMT_CREDIT_SUM_LIMIT      float64
AMT_CREDIT_SUM_OVERDUE    float64
CREDIT_TYPE                object
DAYS_CREDIT_UPDATE          int64
AMT_ANNUITY               float64
dtype: object

In [ ]:
bureau.isnull().sum()

SK_ID_CURR                      0
SK_ID_BUREAU                    0
CREDIT_ACTIVE                   0
CREDIT_CURRENCY                 0
DAYS_CREDIT                     0
CREDIT_DAY_OVERDUE              0
DAYS_CREDIT_ENDDATE        105553
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_MAX_OVERDUE    1124488
CNT_CREDIT_PROLONG              0
AMT_CREDIT_SUM                 13
AMT_CREDIT_SUM_DEBT        257669
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_OVERDUE          0
CREDIT_TYPE                     0
DAYS_CREDIT_UPDATE              0
AMT_ANNUITY               1226791
dtype: int64

In [ ]:
((bureau.select_dtypes('float64').isnull().sum())/bureau.select_dtypes('float64').shape[0])*100

DAYS_CREDIT_ENDDATE        6.149573
DAYS_ENDDATE_FACT         36.916958
AMT_CREDIT_MAX_OVERDUE    65.513264
AMT_CREDIT_SUM             0.000757
AMT_CREDIT_SUM_DEBT       15.011932
AMT_CREDIT_SUM_LIMIT      34.477415
AMT_CREDIT_SUM_OVERDUE     0.000000
AMT_ANNUITY               71.473490
dtype: float64

In [4]:
bureau.fillna(bureau.select_dtypes('float64').mean(), inplace=True)

In [5]:
bureau.isnull().sum()

SK_ID_CURR                0
SK_ID_BUREAU              0
CREDIT_ACTIVE             0
CREDIT_CURRENCY           0
DAYS_CREDIT               0
CREDIT_DAY_OVERDUE        0
DAYS_CREDIT_ENDDATE       0
DAYS_ENDDATE_FACT         0
AMT_CREDIT_MAX_OVERDUE    0
CNT_CREDIT_PROLONG        0
AMT_CREDIT_SUM            0
AMT_CREDIT_SUM_DEBT       0
AMT_CREDIT_SUM_LIMIT      0
AMT_CREDIT_SUM_OVERDUE    0
CREDIT_TYPE               0
DAYS_CREDIT_UPDATE        0
AMT_ANNUITY               0
dtype: int64

In [ ]:
bureau_balance.isnull().sum()

SK_ID_BUREAU      0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

In [6]:
bureau_merge = pd.merge(left=bureau,
                      right=bureau_balance,
                      how="left",
                      left_on="SK_ID_BUREAU",
                      right_index=True)

In [7]:
print(bureau.shape)
print(bureau_balance.shape)
print(bureau_merge.shape)

(1716428, 17)
(27299925, 3)
(1716428, 20)


3. previous_application_merge

In [8]:
credit_card_balance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840312 entries, 0 to 3840311
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   SK_ID_PREV                  int64  
 1   SK_ID_CURR                  int64  
 2   MONTHS_BALANCE              int64  
 3   AMT_BALANCE                 float64
 4   AMT_CREDIT_LIMIT_ACTUAL     int64  
 5   AMT_DRAWINGS_ATM_CURRENT    float64
 6   AMT_DRAWINGS_CURRENT        float64
 7   AMT_DRAWINGS_OTHER_CURRENT  float64
 8   AMT_DRAWINGS_POS_CURRENT    float64
 9   AMT_INST_MIN_REGULARITY     float64
 10  AMT_PAYMENT_CURRENT         float64
 11  AMT_PAYMENT_TOTAL_CURRENT   float64
 12  AMT_RECEIVABLE_PRINCIPAL    float64
 13  AMT_RECIVABLE               float64
 14  AMT_TOTAL_RECEIVABLE        float64
 15  CNT_DRAWINGS_ATM_CURRENT    float64
 16  CNT_DRAWINGS_CURRENT        int64  
 17  CNT_DRAWINGS_OTHER_CURRENT  float64
 18  CNT_DRAWINGS_POS_CURRENT    float64
 19  CNT_INSTALMENT_MATURE

In [ ]:
credit_card_balance.isnull().sum()

SK_ID_PREV                         0
SK_ID_CURR                         0
MONTHS_BALANCE                     0
AMT_BALANCE                        0
AMT_CREDIT_LIMIT_ACTUAL            0
AMT_DRAWINGS_ATM_CURRENT      749816
AMT_DRAWINGS_CURRENT               0
AMT_DRAWINGS_OTHER_CURRENT    749816
AMT_DRAWINGS_POS_CURRENT      749816
AMT_INST_MIN_REGULARITY       305236
AMT_PAYMENT_CURRENT           767988
AMT_PAYMENT_TOTAL_CURRENT          0
AMT_RECEIVABLE_PRINCIPAL           0
AMT_RECIVABLE                      0
AMT_TOTAL_RECEIVABLE               0
CNT_DRAWINGS_ATM_CURRENT      749816
CNT_DRAWINGS_CURRENT               0
CNT_DRAWINGS_OTHER_CURRENT    749816
CNT_DRAWINGS_POS_CURRENT      749816
CNT_INSTALMENT_MATURE_CUM     305236
NAME_CONTRACT_STATUS               0
SK_DPD                             0
SK_DPD_DEF                         0
dtype: int64

In [8]:
credit_card_balance.fillna(credit_card_balance.select_dtypes('float64').mean(), inplace=True)

In [9]:
credit_card_balance.isnull().sum()

SK_ID_PREV                    0
SK_ID_CURR                    0
MONTHS_BALANCE                0
AMT_BALANCE                   0
AMT_CREDIT_LIMIT_ACTUAL       0
AMT_DRAWINGS_ATM_CURRENT      0
AMT_DRAWINGS_CURRENT          0
AMT_DRAWINGS_OTHER_CURRENT    0
AMT_DRAWINGS_POS_CURRENT      0
AMT_INST_MIN_REGULARITY       0
AMT_PAYMENT_CURRENT           0
AMT_PAYMENT_TOTAL_CURRENT     0
AMT_RECEIVABLE_PRINCIPAL      0
AMT_RECIVABLE                 0
AMT_TOTAL_RECEIVABLE          0
CNT_DRAWINGS_ATM_CURRENT      0
CNT_DRAWINGS_CURRENT          0
CNT_DRAWINGS_OTHER_CURRENT    0
CNT_DRAWINGS_POS_CURRENT      0
CNT_INSTALMENT_MATURE_CUM     0
NAME_CONTRACT_STATUS          0
SK_DPD                        0
SK_DPD_DEF                    0
dtype: int64

In [ ]:
POS_CASH_balance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001358 entries, 0 to 10001357
Data columns (total 8 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   SK_ID_PREV             int64  
 1   SK_ID_CURR             int64  
 2   MONTHS_BALANCE         int64  
 3   CNT_INSTALMENT         float64
 4   CNT_INSTALMENT_FUTURE  float64
 5   NAME_CONTRACT_STATUS   object 
 6   SK_DPD                 int64  
 7   SK_DPD_DEF             int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 610.4+ MB


In [ ]:
POS_CASH_balance.isnull().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
MONTHS_BALANCE               0
CNT_INSTALMENT           26071
CNT_INSTALMENT_FUTURE    26087
NAME_CONTRACT_STATUS         0
SK_DPD                       0
SK_DPD_DEF                   0
dtype: int64

In [10]:
POS_CASH_balance.fillna(POS_CASH_balance.select_dtypes('float64').mean(), inplace=True)

In [11]:
POS_CASH_balance.isnull().sum()

SK_ID_PREV               0
SK_ID_CURR               0
MONTHS_BALANCE           0
CNT_INSTALMENT           0
CNT_INSTALMENT_FUTURE    0
NAME_CONTRACT_STATUS     0
SK_DPD                   0
SK_DPD_DEF               0
dtype: int64

In [ ]:
installments_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_PREV              int64  
 1   SK_ID_CURR              int64  
 2   NUM_INSTALMENT_VERSION  float64
 3   NUM_INSTALMENT_NUMBER   int64  
 4   DAYS_INSTALMENT         float64
 5   DAYS_ENTRY_PAYMENT      float64
 6   AMT_INSTALMENT          float64
 7   AMT_PAYMENT             float64
dtypes: float64(5), int64(3)
memory usage: 830.4 MB


In [12]:
installments_payments.isnull().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
NUM_INSTALMENT_VERSION       0
NUM_INSTALMENT_NUMBER        0
DAYS_INSTALMENT              0
DAYS_ENTRY_PAYMENT        2905
AMT_INSTALMENT               0
AMT_PAYMENT               2905
dtype: int64

In [12]:
installments_payments.fillna(installments_payments.select_dtypes('float64').mean(), inplace=True)

In [13]:
installments_payments.isnull().sum()

SK_ID_PREV                0
SK_ID_CURR                0
NUM_INSTALMENT_VERSION    0
NUM_INSTALMENT_NUMBER     0
DAYS_INSTALMENT           0
DAYS_ENTRY_PAYMENT        0
AMT_INSTALMENT            0
AMT_PAYMENT               0
dtype: int64

In [14]:
installments_payments_POS_merge = pd.merge(left=POS_CASH_balance,
                      right=installments_payments,
                      how="inner",
                      left_on="SK_ID_PREV",
                      right_index=True)

In [15]:
print(installments_payments.shape)
print(POS_CASH_balance.shape)
print(installments_payments_POS_merge.shape)

(13605401, 8)
(10001358, 8)
(10001358, 17)


In [16]:
installments_payments_POS_merge.dtypes

SK_ID_PREV                  int64
SK_ID_PREV_x                int64
SK_ID_CURR_x                int64
MONTHS_BALANCE              int64
CNT_INSTALMENT            float64
CNT_INSTALMENT_FUTURE     float64
NAME_CONTRACT_STATUS       object
SK_DPD                      int64
SK_DPD_DEF                  int64
SK_ID_PREV_y                int64
SK_ID_CURR_y                int64
NUM_INSTALMENT_VERSION    float64
NUM_INSTALMENT_NUMBER       int64
DAYS_INSTALMENT           float64
DAYS_ENTRY_PAYMENT        float64
AMT_INSTALMENT            float64
AMT_PAYMENT               float64
dtype: object

In [16]:
installments_POS_credit_merge = pd.merge(left=installments_payments_POS_merge,
                      right=credit_card_balance,
                      how="left",
                      left_on="SK_ID_PREV",
                      right_index=True)

In [17]:
print(credit_card_balance.shape)
print(POS_CASH_balance.shape)
print(installments_payments.shape)
print(installments_POS_credit_merge.shape)

(3840312, 23)
(10001358, 8)
(13605401, 8)
(10001358, 40)


In [18]:
previous_application_merge = pd.merge(left=previous_application,
                      right=installments_POS_credit_merge,
                      how="left",
                      left_on="SK_ID_PREV",
                      right_index=True)

In [19]:
print(previous_application.shape)
print(credit_card_balance.shape)
print(POS_CASH_balance.shape)
print(installments_payments.shape)
print(previous_application_merge.shape)

(1670214, 37)
(3840312, 23)
(10001358, 8)
(13605401, 8)
(1670214, 77)


1. application train/test (bureau and previous_application) <br>
i.e. bureau_merge and previous_application_merge

In [20]:
bureau_pre_application_merge = pd.merge(left=bureau_merge,
                      right=previous_application_merge,
                      how="inner",
                      left_on="SK_ID_CURR",
                      right_index=True)

In [21]:
print(bureau_merge.shape)
print(previous_application_merge.shape)
print(bureau_pre_application_merge.shape)

(1716428, 20)
(1670214, 77)
(1716428, 97)


In [22]:
application_merge = pd.merge(left=train,
                      right=bureau_pre_application_merge,
                      how="inner",
                      left_on="SK_ID_CURR",
                      right_index=True)

In [23]:
print(train.shape)
print(bureau_pre_application_merge.shape)
print(application_merge.shape)

(307511, 122)
(1716428, 97)
(307511, 220)


In [24]:
application_merge.head(10)

,SK_ID_CURR,SK_ID_CURR_x,TARGET,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY,AMT_GOODS_PRICE_x,NAME_TYPE_SUITE_x,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START_x,HOUR_APPR_PROCESS_START_x,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,...,SK_ID_PREV_x,SK_ID_PREV_x,SK_ID_CURR_x,MONTHS_BALANCE_x,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS_x,SK_DPD_x,SK_DPD_DEF_x,SK_ID_PREV_y,SK_ID_CURR_y,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,SK_ID_PREV_y,SK_ID_CURR_y,MONTHS_BALANCE_y,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS_y,SK_DPD_y,SK_DPD_DEF_y
0,100002,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,...,2166066,2166066,203848,-7,12.0,12.0,Active,0,0,1970657,143023,1.0,3,-1034.0,-1054.0,4979.565,4979.565,1375340,446920,-23,0.0,112500,0.0,0.0,0.0,0.0,0.0,207.855,207.855,0.0,0.0,0.0,0.0,0,0.0,0.0,32.0,Active,0,0
1,100003,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,...,2166066,2166066,203848,-7,12.0,12.0,Active,0,0,1970657,143023,1.0,3,-1034.0,-1054.0,4979.565,4979.565,1375340,446920,-23,0.0,112500,0.0,0.0,0.0,0.0,0.0,207.855,207.855,0.0,0.0,0.0,0.0,0,0.0,0.0,32.0,Active,0,0
2,100004,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,...,2166066,2166066,203848,-7,12.0,12.0,Active,0,0,1970657,143023,1.0,3,-1034.0,-1054.0,4979.565,4979.565,1375340,446920,-23,0.0,112500,0.0,0.0,0.0,0.0,0.0,207.855,207.855,0.0,0.0,0.0,0.0,0,0.0,0.0,32.0,Active,0,0
3,100006,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,...,2166066,2166066,203848,-7,12.0,12.0,Active,0,0,1970657,143023,1.0,3,-1034.0,-1054.0,4979.565,4979.565,1375340,446920,-23,0.0,112500,0.0,0.0,0.0,0.0,0.0,207.855,207.855,0.0,0.0,0.0,0.0,0,0.0,0.0,32.0,Active,0,0
4,100007,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,...,2166066,2166066,203848,-7,12.0,12.0,Active,0,0,1970657,143023,1.0,3,-1034.0,-1054.0,4979.565,4979.565,1375340,446920,-23,0.0,112500,0.0,0.0,0.0,0.0,0.0,207.855,207.855,0.0,0.0,0.0,0.0,0,0.0,0.0,32.0,Active,0,0
5,100008,100008,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,454500.0,"Spouse, partner",State servant,Secondary / secondary special,Married,House / apartment,0.035792,-16941,-1588,-4970.0,-477,NaN,1,1,1,1,1,0,Laborers,2.0,2,2,WEDNESDAY,

i was able to merge the file but because of the size of the file processing any thing on the final dataframe the colab server crashes so i would be modelling over the train set instead.

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [31]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

for i in train:
  if train[i].dtype=='object':
        if len(list(train[i].unique()))<=2:
            le.fit(train[i])
            train[i]=le.transform(train[i])
            test[i]=le.transform(test[i])

In [32]:
train=pd.get_dummies(train)
test=pd.get_dummies(test)

In [33]:
x=train.drop(columns=['TARGET'])
y=train['TARGET']

In [35]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.2,random_state =10)

In [52]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

imputer=SimpleImputer(strategy = 'median')

x_train=imputer.fit_transform(x_train)
x_test=imputer.transform(x_test)

scaler=MinMaxScaler(feature_range = (0, 1))
x_train_scale=scaler.fit_transform(x_train)
x_test_scale=scaler.transform(x_test)

In [53]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', random_state=0)
model.fit(x_train,y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [59]:
pred = model.predict(x_train)

In [58]:
model.score(x_train, y_train)

0.9192587232935514

In [61]:
from sklearn.metrics import classification_report
print(classification_report(y_train, pred))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96    226148
           1       0.00      0.00      0.00     19860

    accuracy                           0.92    246008
   macro avg       0.46      0.50      0.48    246008
weighted avg       0.85      0.92      0.88    246008



In [38]:
from sklearn.ensemble import RandomForestClassifier
randomforest=RandomForestClassifier(max_depth=4, random_state=10)
randomforest.fit(x_train_scale,y_train)

from sklearn import metrics
y_pred=randomforest.predict(x_test)
print(metrics.accuracy_score(y_test,y_pred))

0.9192722306228964


In [43]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

dmatrix = xgb.DMatrix(data=x,label=y)

In [44]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,max_depth = 5, alpha = 10, n_estimators = 10)

In [45]:
xg_reg.fit(x_train,y_train)

pred = xg_reg.predict(x_test)

[18:02:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [47]:
import numpy as np
r_mean_square_error = np.sqrt(mean_squared_error(y_test, pred))
print("RMSE: %f" % (r_mean_square_error))

RMSE: 0.305382


In [50]:
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_test,pred)
print(score)

0.7293659012571534
